In [2]:
import pandas as pd
import sys

First, we want to determine which species to use for this analysis: first, those that are present in the photosynthesis pathway dataset and in the chloroplast genome bank

In [24]:
plastids = pd.read_csv('data/taxonomy_info.csv', index_col=False)
print(plastids.shape)

#pathway data is a tsv
phData = pd.read_csv('data/photosynthesisPathways.txt', index_col=0, sep='\t', encoding_errors='replace')
print(phData.shape)

(52758, 5)


/tmp/ipykernel_2214686/208876120.py:5: DtypeWarning: Columns (15,16,18,19,21,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  phData = pd.read_csv('data/photosynthesisPathways.txt', index_col=0, sep='\t', encoding_errors='replace')


(2236552, 28)


In [ ]:
#pathway data is a tsv
phData = pd.read_csv('data/photosynthesisPathways.txt', index_col=0, sep='\t', encoding_errors='replace')
print(phData.shape)

phData = phData[phData["DataName"] == "Plant photosynthetic pathway"]
phData["OrigValueStr"] = phData["OrigValueStr"].replace({
    "c3": "C3", "C3.": "C3", "c4": "C4"
})
clean_phData = phData[phData["OrigValueStr"].isin(["C3", "C4", "CAM"])].copy()

conflicting = (
    clean_phData.groupby("AccSpeciesName")["OrigValueStr"]
    .nunique()
    .reset_index()
    .rename(columns={"OrigValueStr": "n_unique"})
)
conflicting = conflicting[conflicting["n_unique"] > 1]

clean_phData = clean_phData[~clean_phData["AccSpeciesName"].isin(conflicting["AccSpeciesName"])]

toMerge = clean_phData[["AccSpeciesName", "OrigValueStr"]].drop_duplicates()

In [26]:
#now, we will attempt to merge this with the plastid data

#now, determine the species binomial from the plastid Organism column - take the first two words
plastids["species_name"] = plastids["Organism"].apply(lambda x: " ".join(x.split()[:2]) if isinstance(x, str) else x)
print(plastids.columns)

merged = pd.merge(toMerge, plastids, left_on="AccSpeciesName", right_on="species_name", how="inner")

#keep only species with a pathway annotated
merged = merged[merged["OrigValueStr"].notnull()]

#how many species have data from the pathway?
print(merged.shape)
print(merged.head())

Index(['ID', 'Organism', 'Taxonomy', 'Year', 'SequencingTech', 'species_name'], dtype='object')
(19602, 8)
         AccSpeciesName OrigValueStr           ID              Organism  \
0        Holcus lanatus           C3  NC_036689.1        Holcus lanatus   
1        Holcus lanatus           C3   KY432781.1        Holcus lanatus   
2  Agrostis stolonifera           C3   OQ695464.1  Agrostis stolonifera   
3  Agrostis stolonifera           C3  NC_008591.1  Agrostis stolonifera   
4  Agrostis stolonifera           C3   EF115543.1  Agrostis stolonifera   

                                            Taxonomy  Year SequencingTech  \
0  Eukaryota; Viridiplantae; Streptophyta; Embryo...  2023        Unknown   
1  Eukaryota; Viridiplantae; Streptophyta; Embryo...  2017        Unknown   
2  Eukaryota; Viridiplantae; Streptophyta; Embryo...  2023        Unknown   
3  Eukaryota; Viridiplantae; Streptophyta; Embryo...  2023        Unknown   
4  Eukaryota; Viridiplantae; Streptophyta; Embryo...  201

In [ ]:
#how many unique species are there, by AccSpeciesName?
print(merged["AccSpeciesName"].nunique())

#keep only one record for unique species, and decide by the most recent record by Year - if there is a tie, 
merged = merged.sort_values(by="Year", ascending=False).drop_duplicates(subset=["AccSpeciesName"])

#what is the distribution of photosynthesis types?
print(merged["OrigValueStr"].value_counts())

#make a LUI list to use for the genome analysis - combine the binomial (species_name) and the ID, removing all non-alphanumeric characters
merged["LUI"] = merged["ID"].str.replace(r'[^a-zA-Z0-9]', '', regex=True) + merged["species_name"].str.replace(r'[^a-zA-Z0-9]', '', regex=True)
#save the list of luis to a file called pathway_luis.txt

4691
OrigValueStr
C3     4371
C4      264
CAM      56
Name: count, dtype: int64


In [46]:
#All of these should map to a chloroplast genome in data/genomes
#double check each LUI is a file in data/genomes
import os
i=0
for lui in merged["LUI"]:
    if not os.path.exists(f"data/genomes/{lui}.fa"):
        i +=1
        #drop that row from merged
        merged = merged[merged["LUI"] != lui]
        #print(f"Missing genome for {lui}")
print(i)
merged[["LUI"]].to_csv("data/pathway_luis.txt", index=False, header=False)

merged.to_csv("data/pathway_species.csv", index=False)

0


In [ ]:
#./countMers.sh, let's look at 6mers

In [48]:
import pandas as pd
import glob
import os
import re

indir = "jellyfish_output"
files = glob.glob(os.path.join(indir, "k*/**/*_counts.tsv"), recursive=True)

dfs = []

for f in files:
    # Extract LUI and k-mer size from the path
    match = re.search(r"k(\d+)/(.+?)_counts\.tsv", f)
    if not match:
        continue
    ksize, genome = match.groups()
    genome_label = f"{genome}_k{ksize}"

    # Read and rename column with genome and k info
    df = pd.read_csv(f, sep=r"\s+", header=None, names=["kmer", genome_label])
    dfs.append(df.set_index("kmer"))

# Combine and fill missing values with 0
kmer_matrix = pd.concat(dfs, axis=1).fillna(0).astype(int).sort_index()

# Transpose so genomes are rows, kmers are columns
kmer_matrix = kmer_matrix.T

# Save final matrix
kmer_matrix.to_csv("data/kmer_matrix.tsv", sep="\t")

print(kmer_matrix.shape)
print(kmer_matrix.head())


(54, 367945)
kmer                                  AA  AAA  AAAA  AAAAA  AAAAAA  AAAAAAA  \
PV4402251Euphorbianeriifolia_k2    39527    0     0      0       0        0   
PP2345351Lactucatatarica_k2        33764    0     0      0       0        0   
PQ8691531Cercocarpusledifolius_k2  35898    0     0      0       0        0   
PP6396921Heveabrasiliensis_k2      38099    0     0      0       0        0   
PP2346001Hippurisvulgaris_k2       34216    0     0      0       0        0   

kmer                               AAAAAAAA  AAAAAAAAA  AAAAAAAAAA  \
PV4402251Euphorbianeriifolia_k2           0          0           0   
PP2345351Lactucatatarica_k2               0          0           0   
PQ8691531Cercocarpusledifolius_k2         0          0           0   
PP6396921Heveabrasiliensis_k2             0          0           0   
PP2346001Hippurisvulgaris_k2              0          0           0   

kmer                               AAAAAAAAAC  ...  TTTTACAAAA  TTTTAGAAAA  \
PV4402251Euph